<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Estadistica_Piez%C3%B3metros_El%C3%A9ctricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🔍 FASE 1 — Estadística Descriptiva: Piezómetros Eléctricos

##🎯 Objetivo de esta etapa:
Obtener una comprensión clara de las variables medidas por los piezómetros eléctricos a través de estadísticas descriptivas y visualizaciones.

In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'PE175_20250608.csv': {'metadata': {'name': 'PE175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PIEZOMETROS_ELECTRICOS (XLSX)


,FECHA,PROGRESIVA,PIEZOMETRO,COTA_RIO_(M.S.N.M),LECTURA_CUERDA_VIBRANTE,TEMPERATURA_(°C),MCA_1_(FACTOR_G),MCA_2_(FACTOR_G_Y_K),COTA_NF,PRECIPITACIONES_(MM)
0,18/07/2023,0+175,PE 7,115.156,8938.93,2.7,0.030356,0.054266,119.836356,NaN
1,29/07/2023,0+175,PE 7,114.831,8911.57,0.9,0.806559,0.859161,120.612559,NaN
2,15/08/2023,0+175,PE 7,114.629,8939.79,1.5,0.005958,0.048996,119.811958,NaN
3,24/08/2023,0+175,PE 7,114.391,8949.63,1.5,-0.273203,-0.230165,SIN NF,3.8
4,01/09/2023,0+175,PE 7,114.309,8953.65,1.6,-0.387250,-0.345806,SIN NF,NaN


In [16]:
# === IMPORTACIONES GENERALES ===

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.io as pio
from IPython.display import display, clear_output, Markdown
import ipywidgets as widgets

pio.renderers.default = 'colab'

# === SUPONEMOS QUE YA TENÉS CARGADOS ESTOS DICCIONARIOS ===
# datos_csv["piezometros_electricos"]
# datos_xlsx["piezometros_electricos"]

# === PASO 1: Selección de origen, progresiva y piezómetro ===

origen_selector = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
progresiva_selector = widgets.Dropdown(options=[], description="Progresiva:")
piezometro_selector = widgets.Dropdown(options=[], description="Piezómetro:")
boton_mostrar = widgets.Button(description="Mostrar datos", button_style='success')
salida_datos = widgets.Output()

def actualizar_progresivas(change=None):
    origen = origen_selector.value
    df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_selector.options = sorted(df['PROGRESIVA'].dropna().unique())

def actualizar_piezometros(change=None):
    origen = origen_selector.value
    df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
    df = df[df["PROGRESIVA"] == progresiva_selector.value]
    piezometro_selector.options = sorted(df["PIEZOMETRO"].dropna().unique())

def mostrar_datos(b):
    with salida_datos:
        clear_output()
        origen = origen_selector.value
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df[(df["PROGRESIVA"] == progresiva_selector.value) & (df["PIEZOMETRO"] == piezometro_selector.value)]
        display(Markdown(f"📌 Datos seleccionados — Origen: **{origen}**, Progresiva: **{progresiva_selector.value}**, Piezómetro: **{piezometro_selector.value}**"))
        display(df.head())

origen_selector.observe(actualizar_progresivas, names="value")
progresiva_selector.observe(actualizar_piezometros, names="value")
boton_mostrar.on_click(mostrar_datos)

display(Markdown("## 🔍 Paso 1 – Selección de datos"))
display(widgets.HBox([origen_selector, progresiva_selector, piezometro_selector, boton_mostrar]))
display(salida_datos)

actualizar_progresivas()
actualizar_piezometros()

# === PASO 2: Estadística Descriptiva ===

variable_selector = widgets.Dropdown(description="Variable:")
boton_analizar = widgets.Button(description="Analizar variable", button_style="info")
salida_estadisticas = widgets.Output()

def actualizar_variables(change=None):
    origen = origen_selector.value
    df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
    df = df[(df["PROGRESIVA"] == progresiva_selector.value) & (df["PIEZOMETRO"] == piezometro_selector.value)]
    num_vars = df.select_dtypes(include=[np.number]).columns.tolist()
    if 'COTA_NF' in df.columns and 'COTA_NF' not in num_vars:
        df['COTA_NF'] = pd.to_numeric(df['COTA_NF'], errors='coerce')
        if df['COTA_NF'].notna().any():
            num_vars.append('COTA_NF')
    variable_selector.options = num_vars
    if num_vars:
        variable_selector.value = num_vars[0]

def analizar_variable(b):
    with salida_estadisticas:
        clear_output()
        origen = origen_selector.value
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df[(df["PROGRESIVA"] == progresiva_selector.value) & (df["PIEZOMETRO"] == piezometro_selector.value)]
        var = variable_selector.value
        df[var] = pd.to_numeric(df[var], errors='coerce')
        df = df.dropna(subset=[var])
        valores = df[var]
        display(Markdown(f"### 📊 Estadísticas de `{var}`"))
        display(pd.DataFrame({
            'Media': [valores.mean()],
            'Mediana': [valores.median()],
            'Moda': [valores.mode().iloc[0] if not valores.mode().empty else np.nan],
            'Desv. Estándar': [valores.std()],
            'Varianza': [valores.var()],
            'IQR': [stats.iqr(valores)],
            'CV': [valores.std()/valores.mean()]
        }).T.rename(columns={0: 'Valor'}))
        fig, axs = plt.subplots(1, 2, figsize=(12, 4))
        sns.histplot(valores, ax=axs[0], kde=True, color='skyblue')
        axs[0].set_title("Histograma")
        sns.boxplot(x=valores, ax=axs[1], color='salmon')
        axs[1].set_title("Boxplot")
        plt.tight_layout()
        plt.show()

progresiva_selector.observe(actualizar_variables, names="value")
piezometro_selector.observe(actualizar_variables, names="value")
origen_selector.observe(actualizar_variables, names="value")
boton_analizar.on_click(analizar_variable)

display(Markdown("## 📈 Paso 2 – Estadística descriptiva"))
display(widgets.HBox([variable_selector, boton_analizar]))
display(salida_estadisticas)

# === PASO 3: Regresión lineal + Predicción futura ===

dias_futuros_selector = widgets.IntSlider(value=30, min=7, max=365, step=7, description='Días a predecir:')
boton_regresion = widgets.Button(description="Aplicar Regresión", button_style="warning")
salida_regresion = widgets.Output()

def aplicar_regresion(b):
    with salida_regresion:
        clear_output()
        origen = origen_selector.value
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
        var = variable_selector.value
        piezo = piezometro_selector.value
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df[(df["PROGRESIVA"] == progresiva_selector.value) & (df["PIEZOMETRO"] == piezo)]
        df[var] = pd.to_numeric(df[var], errors='coerce')
        df = df.dropna(subset=['FECHA', var])
        df['FECHA_NUM'] = df['FECHA'].map(lambda x: x.toordinal())
        X = df[['FECHA_NUM']]
        y = df[var]
        modelo = LinearRegression().fit(X, y)
        y_pred = modelo.predict(X)
        future_dates = pd.date_range(start=df['FECHA'].max() + pd.Timedelta(days=1), periods=dias_futuros_selector.value)
        future_ordinals = future_dates.map(lambda x: x.toordinal()).values.reshape(-1, 1)
        y_future = modelo.predict(future_ordinals)

        display(Markdown(f"### 📉 Regresión Lineal para `{var}`"))
        display(Markdown(f"Coef: `{modelo.coef_[0]:.4f}`, Intercepto: `{modelo.intercept_:.2f}`, R²: `{r2_score(y, y_pred):.4f}`"))

        plt.figure(figsize=(10, 4))
        plt.scatter(df['FECHA'], y, color='blue', label='Datos')
        plt.plot(df['FECHA'], y_pred, color='red', label='Modelo')
        plt.plot(future_dates, y_future, color='green', linestyle='--', label='Predicción')
        plt.xticks(rotation=45)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        pred_df = pd.DataFrame({'Fecha': future_dates, f'Predicción {var}': y_future})
        display(Markdown("### 🔮 Predicción futura"))
        display(pred_df.head(10))

boton_regresion.on_click(aplicar_regresion)

display(Markdown("## 📐 Paso 3 – Regresión Lineal + Predicción"))
display(dias_futuros_selector)
display(boton_regresion)
display(salida_regresion)

# === PASO 4: Clustering de piezómetros ===

cluster_slider = widgets.IntSlider(value=3, min=2, max=10, step=1, description='Clusters:')
boton_cluster = widgets.Button(description="Aplicar Clustering", button_style='primary')

def aplicar_clustering(b):
    clear_output()
    origen = origen_selector.value
    df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variables = ['COTA_NF', 'PRESION', 'CARGA']
    columnas_existentes = [v for v in variables if v in df.columns]
    for col in columnas_existentes:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=['PIEZOMETRO'] + columnas_existentes)
    agrupado = df.groupby("PIEZOMETRO")[columnas_existentes].mean().dropna()
    if agrupado.shape[0] < cluster_slider.value:
        print("⚠️ Menos piezómetros que clusters")
        return
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(agrupado)
    if X_scaled.shape[1] >= 2:
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        df_vis = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
    else:
        df_vis = pd.DataFrame(X_scaled, columns=["PC1"])
        df_vis["PC2"] = 0
    df_vis["PIEZOMETRO"] = agrupado.index
    kmeans = KMeans(n_clusters=cluster_slider.value, n_init=10, random_state=0)
    df_vis["Cluster"] = kmeans.fit_predict(X_scaled).astype(str)

    display(Markdown("## 🧠 Paso 4 – Clustering de Piezómetros"))
    fig = px.scatter(df_vis, x="PC1", y="PC2", color="Cluster", text="PIEZOMETRO",
                     title="📊 Clustering por Comportamiento Promedio", width=800, height=500)
    fig.update_traces(textposition='top center')
    fig.show()

    display(Markdown("### 📋 Tabla de Clusters"))
    display(df_vis.sort_values("Cluster").reset_index(drop=True))

boton_cluster.on_click(aplicar_clustering)

display(Markdown("## 🧠 Paso 4 – Clustering"))
display(cluster_slider)
display(boton_cluster)


## 🧠 Paso 4 – Clustering de Piezómetros

### 📋 Tabla de Clusters

,PC1,PC2,PIEZOMETRO,Cluster
0,1.589932,0,PE 3,0
1,1.582964,0,PE 4,0
2,1.565018,0,PE 5,0
3,-0.917299,0,PE 10,1
4,-0.962071,0,PE 11,1
5,-0.987823,0,PE 12,1
6,-0.781643,0,PE 13,1
7,-0.937312,0,PE 14,1
8,-0.255534,0,PE 6,2
9,0.104024,0,PE 7,2
